In [2]:
import re

class Morph():
    def __init__(self, doc_list):
        self.surface = doc_list[0]
        self.base = doc_list[7]
        self.pos = doc_list[1]
        self.pos1= doc_list[2]

    def __str__(self):
        return "{" + f"surface: {self.surface}, base: {self.base}, pos: {self.pos}, pos1: {self.pos1}" + "}"

class Chunk():
    def __init__(self,morph_list,dst,srcs):
        self.morph_list = morph_list
        self.dst = int(dst)
        self.srcs = srcs

    def __str__(self):
        morphs_str = ', '.join(str(morph) for morph in self.morph_list) # Morphオブジェクトのリストを文字列に変換
        return f"Morphs: [{morphs_str}], Dst: {self.dst}, Srcs: {self.srcs}"

# 文節番号と係り先番号の取得
def find_feature_number(text):
    pattern = r"^\* \d+ ([^\d]*\d+)D"
    match = re.search(pattern, text)
    if match:
        dst_number = match.group(1)  # 文節番号  # 係先番号
        return dst_number
    else:
        return None

In [3]:
with open(file="ai.ja.txt.parsed", mode="r", encoding="utf-8")as f:
    docs = f.readlines()

Chunk_list = []
chunk_flag = False
# srcの追加処理は後で行う
for doc in docs[5:]:
    if doc == "EOS\n":
        chunk = Chunk(morph_list,dst_num,[])
        Chunk_list.append(chunk)
        break
    # *から始まる行とEOSの行は飛ばす
    if doc[0]=="*":
        if chunk_flag:
            chunk = Chunk(morph_list,dst_num,[])
            Chunk_list.append(chunk)
        dst_num = find_feature_number(doc)
        morph_list = []
        chunk_flag = True
    else:
        doc = doc.replace("\t",",").replace("\n", "")
        doc_list = doc.split(",")
        doc_morph = Morph(doc_list)
        morph_list.append(doc_morph)


# Srcsを埋めていく
for i in range(len(Chunk_list)):
    Chunk_list[Chunk_list[i].dst].srcs.append(i)

In [30]:
#名詞を含む場合はchunkを返す，なければNone
def get_noun_sentence(chunk: Chunk):
    noun_flag = False
    for morph in chunk.morph_list:
        if morph.pos == "名詞":
            noun_flag = True
    if noun_flag:
        return chunk
    return None

#chunkから表層形の文章を再構築
def print_sentence_from_chunk(chunk):
    remove_list = ["（", "）", "、", "。", "「", "」", "『", "』", "〈", "〉"]
    sentence = ""
    for morph in chunk.morph_list:
        if morph.surface not in remove_list:
            sentence += morph.surface
    print(sentence, end="")

#現在のchunkから根を遡っていく
def print_root_path(chunk: Chunk, chunk_list: list):
    if chunk.dst != -1:
        print_sentence_from_chunk(chunk)
        print("->", end = "")
        print_root_path(chunk_list[chunk.dst], chunk_list)
    else:
        print_sentence_from_chunk(chunk)
        print() #根まで来たら改行

for chunk in Chunk_list:
    has_noun_in_chunk = get_noun_sentence(chunk)
    if has_noun_in_chunk != None:
        print_root_path(chunk, Chunk_list)

人工知能->語->研究分野とも->される
じんこうちのう->語->研究分野とも->される
AI->エーアイとは->語->研究分野とも->される
エーアイとは->語->研究分野とも->される
計算->という->道具を->用いて->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
概念と->道具を->用いて->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
コンピュータ->という->道具を->用いて->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
道具を->用いて->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
知能を->研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
研究する->計算機科学->の->一分野を->指す->語->研究分野とも->される
計算機科学->の->一分野を->指す->語->研究分野とも->される
一分野を->指す->語->研究分野とも->される
語->研究分野とも->される
言語の->推論->問題解決などの->知的行動を->代わって->行わせる->技術または->研究分野とも->される
理解や->推論->問題解決などの->知的行動を->代わって->行わせる->技術または->研究分野とも->される
推論->問題解決などの->知的行動を->代わって->行わせる->技術または->研究分野とも->される
問題解決などの->知的行動を->代わって->行わせる->技術または->研究分野とも->される
知的行動を->代わって->行わせる->技術または->研究分野とも->される
人間に->代わって->行わせる->技術または->研究分野とも->される
コンピューターに->行わせる->技術または->研究分野とも->される
技術または->研究分野とも->される
計算機->コンピュータによる->情報処理システムの->実現に関する->研究分野とも->される
コンピュータによる->情報処理システムの->実現に関する->研究分野とも->される
知的な->情報処理システムの->実現に関する->研究分野とも->される
情報処理システムの->実現に関する->研究分野とも->される
設計や->実現に関する->研究分野と